In [ ]:
import pandas as pd
from pmdarima import auto_arima
from joblib import Parallel, delayed  # Para paralelizar (opcional)
import warnings
warnings.filterwarnings('ignore')  # Ignorar warnings

In [ ]:
productos_ok = pd.read_csv("https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/product_id_apredecir201912.txt", sep="\t")

df = pd.read_csv("./datasets/sell-in.csv", sep="\t")
df.shape

In [ ]:
ts = df.groupby(["periodo", "product_id"]).agg({"tn": "sum"}).reset_index()

ts = ts[ts['product_id'].isin(productos_ok['product_id'])]

In [ ]:
# Convertir 'periodo' (yyyymm) a datetime
ts['periodo_dt'] = pd.to_datetime(ts['periodo'].astype(str), format='%Y%m')
ts = ts.sort_values(['product_id', 'periodo_dt'])  # Ordenar por producto y fecha
ts

In [ ]:
# Convertir 'periodo' (yyyymm) a datetime y ordenar
ts['periodo_dt'] = pd.to_datetime(ts['periodo'].astype(str), format='%Y%m')
ts = ts.sort_values(['product_id', 'periodo_dt'])

# Diccionario para almacenar predicciones
predictions = {}

# Iterar sobre cada producto
for product_id in productos_ok['product_id'].unique():
    # Filtrar datos del producto actual
    df_product = ts[ts['product_id'] == product_id].copy()
    
    # Configurar serie temporal correctamente
    df_product = df_product.set_index('periodo_dt').asfreq('MS')
    
    # Verificar que hay suficientes datos
    if len(df_product) >= 12:
        try:
            # Ajustar AutoARIMA con parámetros estacionales adecuados
            model = auto_arima(
                df_product['tn'],
                seasonal=True,
                m=12,  # Estacionalidad anual para datos mensuales
                suppress_warnings=True,
                error_action='ignore',
                trace=True  # Para ver el proceso de ajuste
            )
            
            # Predecir 2 meses adelante
            pred = model.predict(n_periods=2)
            predictions[product_id] = pred[-1]  # Tomar el segundo mes
            
            print(f"Producto {product_id}: Modelo {model} ajustado")
            
        except Exception as e:
            print(f"Error en producto {product_id}: {str(e)}")
            predictions[product_id] = None
    else:
        print(f"Producto {product_id} no tiene suficientes datos ({len(df_product)} observaciones)")
        predictions[product_id] = None

# Convertir a DataFrame
df_predictions = pd.DataFrame({
    'product_id': predictions.keys(),
    'prediccion_mes+2': predictions.values()
})

print("\nPredicciones finales:")
print(df_predictions)

In [ ]:
df_predictions.isna().sum()

In [ ]:
df_predictions.head()

In [ ]:
df_predictions_copy = df_predictions.copy()
df_predictions_copy

In [ ]:
df_predictions_copy.rename(columns={'prediccion_mes+2': 'tn'}, inplace=True)

In [ ]:
df_predictions_copy.fillna(0, inplace=True)  # Reemplazar NaN por 0
df_predictions_copy.to_csv("../../outputs/predicciones_autoarima.csv", index=False)

In [ ]:
df_predictions_copy[df_predictions_copy['tn'] < 0]

In [ ]:
df_predictions_copy.loc[df_predictions_copy['tn'] < 0, 'tn'] = 0
df_predictions_copy.to_csv("./kaggle/predicciones_autoarima_sin_negativos.csv", index=False)